In [1]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [3]:
x_train, y_train, x_test_original, y_test_original = ReadTS('ACSF1')
#y_train = y_train - 1
#y_test_original = y_test_original - 1
print(x_train.shape)

(100, 1460, 1)


In [4]:
from sklearn.preprocessing import StandardScaler

# 将数据集reshape成二维数组
x_train_2d = x_train.reshape(100, 1460)

# 初始化StandardScaler
scaler = StandardScaler()

# 对数据集进行标准化
x_train_normalized = scaler.fit_transform(x_train_2d)

# 将标准化后的数据集reshape回原始形状
x_train = x_train_normalized.reshape(100, 1460, 1)

In [5]:
model = make_CNN_model(x_train.shape[1:],num_classes = 10)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


2024-11-14 20:09:46.462467: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 1s 107ms/step - loss: 2.2954 - sparse_categorical_accuracy: 0.1750 - val_loss: 2.3112 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 0s 44ms/step - loss: 1.9629 - sparse_categorical_accuracy: 0.4375 - val_loss: 2.3228 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 0s 45ms/step - loss: 1.7999 - sparse_categorical_accuracy: 0.5000 - val_loss: 2.3300 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 0s 49ms/step - loss: 1.6837 - sparse_categorical_accuracy: 0.5375 - val_loss: 2.3348 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 0s 50ms/step - loss: 1.5817 - sparse_categorical_accuracy: 0.6000 - val_loss: 2.3389 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/100
3/3 [==============================] - 0s 48ms/step - loss: 1.4791 - sparse_categorical_a

In [6]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

4/4 [==============================] - 0s 8ms/step - loss: 3.4758 - sparse_categorical_accuracy: 0.0600
Test accuracy 0.05999999865889549
Test loss 3.475785255432129


### Create perturbed test sets and calculate decrease in accuracy

In [7]:
print(y_test_original.shape)

(100,)


In [8]:
print(inspect.getsource(perturb))
print(inspect.signature(perturb))

def perturb(perturbation_strategy, ts, index0, index1, global_ts = []):
    if perturbation_strategy == 'RBP':
        return RBPIndividual(ts, index0, index1)
    if perturbation_strategy == 'zero':
        return zeroPerturb(ts, index0, index1)
    if perturbation_strategy == 'noise':
        return noisePerturb(ts, index0, index1)
    if perturbation_strategy == 'blur':
        return blurPerturb(ts, index0, index1)
    if perturbation_strategy == 'RBP1':
        return RBPIndividualNew1(global_ts, ts, index0, index1)    
    if perturbation_strategy == 'RBP2':
        return RBPIndividualNew2(global_ts, ts, index0, index1)    

(perturbation_strategy, ts, index0, index1, global_ts=[])


In [17]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])

x_test_rbp = np.asarray([perturb('RBP',x,60,100) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

x_test_new1 = np.asarray([perturb('RBP',x,60,100, global_ts = perturb_test_set) for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,60,100) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

x_test_noise = np.asarray([perturb('noise',x,60,100) for x in perturb_test_set])
x_test_noise = np.expand_dims(x_test_noise, axis=-1)
predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,60,100) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

4/4 [==============================] - 0s 7ms/step


In [18]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

In [19]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [20]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    " Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.06
 RBP Accuracy: 0.06
 RBP1 Accuracy: 0.06
 Zero Accuracy: 0.06
 Noise Accuracy: 0.06
 Blur Accuracy: 0.06



#### The results above shows that the two improved perturbations can lower the accuracy of classification to the same level as the original perturbation approach, indicating that these backgrounds are equally uninformative.

## Test Realism of Generated Samples 

### Generate synthetic data 

In [ ]:
perturbed_datasets = generateSyntheticPerturbReal(500)
def get_list_shape(lst):
    if isinstance(lst, list):
        return [len(lst)] + get_list_shape(lst[0]) if lst else []
    else:
        return []

# 示例列表
print(get_list_shape(perturbed_datasets))  # 输出: [2, 3]

In [ ]:
print(inspect.getsource(generateSyntheticPerturbReal))
print(inspect.signature(generateSyntheticPerturbReal))
print(inspect.getfile(generateSyntheticPerturbReal))

### Create CNN model and train on each perturbed dataset 

In [ ]:
datasets_training_loss = []
for dataset in perturbed_datasets: 
    model = make_CNN_model(dataset[0].shape[1:])
    trained_model, history = train_CNN_model(model, dataset[0], dataset[1], epochs=100, batch_size=32)
    datasets_training_loss.append(history.history['val_loss'])

### Visualise loss curves of model training on perturbed datasets

In [ ]:

for loss in datasets_training_loss:
    plt.plot(loss)
plt.title('Model Loss Under Different Perturbations')
plt.ylabel('val loss')
plt.xlabel('epoch')
plt.legend(['RBP','RBP1','RBP2','zero','random','blur'], loc='upper left')
plt.savefig("realistic_perturb.eps", format='eps')
plt.show()

#### The plot above shows that the new perturbations cannot stop the loss from descending over epoch, indicating that the new backgrounds are identifiable. This is predictable to some extent, because all samples share the same background unlike the old RBP method. To prove our method meaningful, maybe a new reasoning together with a new evaluation process is needed, as the concept of 'realistic' may not necessarily mean 'all samples does not share the same background'.